In [ ]:
# Install Kaggle (if dataset not uploaded)
!pip install -q kaggle

# Upload kaggle.json to Colab before running this cell
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d uciml/sms-spam-collection-dataset -p data --unzip

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
# Load dataset
sms = pd.read_csv('data/spam.csv', encoding='latin-1')[['v1', 'v2']]
sms.columns = ['label', 'text']
sms['label'] = sms['label'].map({'ham': 0, 'spam': 1})

In [ ]:
# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(sms.text)
sequences = tokenizer.texts_to_sequences(sms.text)
padded = pad_sequences(sequences, maxlen=100, padding='post')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded, sms.label, test_size=0.2, random_state=42)

In [ ]:
# CNN Model
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
# Training
model.fit(X_train, y_train, epochs=5, validation_split=0.1, batch_size=32)

In [ ]:
# Evaluation
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))

In [ ]:
# Save model
model.save('../models/cnn_spam.h5')